In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np

import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt
import defmod.hamiltonian_multishape as hamiltonian
import model_graddesc as model
import defmod.registrationdata as regdata

torch.set_default_tensor_type(torch.DoubleTensor)

In [2]:
source_vertices = torch.tensor([[1.,3.],[5.,5.],[8., 3.]])
target_vertices = torch.tensor([[1.,5.],[5.,4.],[8., 5.]])
nb_pts = [10,4]

data = regdata.part_rigid(source_vertices, target_vertices, nb_pts)
data.build_source()
data.build_target()
data.build_modules()
source = data.source
target = data.target

In [ ]:
# Visualization of the source shape
# The shape is defined by the points at the black crosses

x = data.source[0].clone()
y = data.source[1].clone()

plt.figure()
plt.scatter(x[:, 0].detach().numpy(), x[:, 1].detach().numpy(), c='red')
plt.scatter(y[:, 0].detach().numpy(), y[:, 1].detach().numpy(), c='blue')
plt.scatter(source_vertices[:, 0].detach().numpy(), source_vertices[:, 1].detach().numpy(), c='k', marker='x')

plt.axis('equal')
plt.show()

In [ ]:
# Visualization of the source and target
plt.figure()
plt.scatter(source[0][:, 0].detach().numpy(), source[0][:, 1].detach().numpy(), c='red', marker='x')
plt.scatter(source[1][:, 0].detach().numpy(), source[1][:, 1].detach().numpy(), c='blue', marker='+')
plt.axis('equal')

plt.figure()
plt.scatter(target[0][:, 0].detach().numpy(), target[0][:, 1].detach().numpy(), c='red', marker='x')
plt.scatter(target[1][:, 0].detach().numpy(), target[1][:, 1].detach().numpy(), c='blue', marker='+')



plt.axis('equal')
plt.show()

In [3]:
# Definition of the multi shape module and Hamiltonian
modules = dm.multishape.MultiShapeModule([data.modules[0], data.modules[1]], sigma_background=0.1, reduce_background=True)
modules.fill_controls_zero()
constr_Id = dm.constraints.Identity_Silent_reduced()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [4]:
modules.manifold[0][0].fill_cotan(torch.ones_like(modules.manifold.cotan[0][0]))

In [ ]:
a, b, c, d = modules.manifold[0].cotan

In [ ]:
ee = modules.manifold[0].cotan

In [ ]:
modules.manifold[0].fill_cotan([torch.ones_like(a), b, c, d])

In [ ]:
modules.manifold[0][0].cotan

In [ ]:
aa = torch.ones_like(a)

In [ ]:
modules.manifold[0].fill_cotan([aa, b, c, d])

In [ ]:
torch.ones_like(modules.manifold.cotan[0][0])

In [ ]:
fields[2]

In [ ]:
# Definition of the similarity metric
from defmod.attachement import L2NormAttachement

def attach(modules, target):
    return sum([L2NormAttachement()( modules.module_list[i][-1].manifold.gd.view(-1,2), target[i]) for i in range(len(target))])

In [5]:
gd0 = modules.module_list[0].manifold[0].gd
gd1 = modules.module_list[1].manifold[0].gd
gd2 = modules.module_list[-1].manifold.gd[0]

gd_silent = torch.cat([gd0,gd1,gd2]).view(-1,1)

In [6]:
H.geodesic_controls()

In [7]:
print(modules.controls)

[[tensor([], requires_grad=True), tensor([ 6.4801e-01,  6.4801e-01,  6.6677e-01,  6.6677e-01,  6.9250e-01,
         6.9250e-01, -5.5935e-17, -5.5935e-17,  5.5265e-17,  5.5265e-17,
        -2.9255e-17, -2.9255e-17,  1.5694e-17,  1.5694e-17, -8.3304e-18,
        -8.3304e-18,  4.3566e-18,  4.3566e-18, -2.2514e-18, -2.2514e-18,
         1.1074e-18,  1.1074e-18, -4.2194e-19, -4.2194e-19, -2.4891e-21,
        -2.4891e-21,  1.4537e-21,  1.4537e-21, -2.0068e-17, -2.0068e-17,
         5.4894e-17,  5.4894e-17, -1.0095e-16, -1.0095e-16,  1.5808e-16,
         1.5808e-16, -2.2461e-16, -2.2461e-16,  2.9048e-16,  2.9048e-16,
        -3.2360e-16, -3.2360e-16,  2.9161e-16,  2.9161e-16, -3.2526e-16,
        -3.2526e-16,  6.9550e-01,  6.9550e-01], grad_fn=<ViewBackward>), tensor([], requires_grad=True)], [tensor([], requires_grad=True), tensor([ 7.6535e-01,  7.6535e-01, -1.6868e-15, -1.6868e-15,  2.3854e-15,
         2.3854e-15, -2.5405e-15, -2.5405e-15,  2.3138e-15,  2.3138e-15,
        -1.8760e-15, -1.

In [8]:
fields = modules.field_generator().fieldlist
        
action = torch.cat([ f(man.manifold_list[0].gd.view(-1,2)) for f,man in zip(fields, modules.manifold.manifold_list)]).view(-1,1)

In [9]:
action.view(-1, 8)

tensor([[1.0691, 1.0691, 1.0813, 1.0813, 0.9013, 0.9013, 0.8988, 0.8988],
        [1.0599, 1.0599, 1.0964, 1.0964, 0.9532, 0.9532, 0.9583, 0.9583],
        [0.7380, 0.7380, 0.6566, 0.6566, 0.5659, 0.5659, 0.5392, 0.5392]],
       grad_fn=<ViewBackward>)

In [ ]:
modules.background.manifold[0].gd

In [10]:
dm.constraints.Identity_Silent_reduced().call_by_matmul(modules)

tensor([[0.3312],
        [0.3312],
        [0.4247],
        [0.4247],
        [0.3354],
        [0.3354],
        [0.3596],
        [0.3596],
        [0.3219],
        [0.3219],
        [0.4399],
        [0.4399],
        [0.3873],
        [0.3873],
        [0.4192],
        [0.4192]], grad_fn=<MmBackward>)

In [ ]:
dm.constraints.Identity_Silent_reduced()(modules).view(-1)

In [ ]:
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
mom0_tensor = torch.cat(H.module.manifold.unroll_cotan().copy(),0).requires_grad_().view(-1)
    
gd0_tensor.retain_grad()
mom0_tensor.retain_grad()
    
EnergyFunctional = model.EnergyFunctional(modules, H, constr_Id,source, target, 2, gamma=10**-3, attach=attach)


In [ ]:
def fun(P, *args):
    #EnergyFunctional = args[0]
    Y = torch.tensor(P).requires_grad_()
    return EnergyFunctional.energy_tensor(gd0_tensor, Y).detach().numpy()[0]

In [ ]:
def jac(P):
    #EnergyFunctional = args[0]
    Y = torch.tensor(P).requires_grad_()
    return EnergyFunctional.gradE_autograd(gd0_tensor, Y).detach().numpy()

In [ ]:
import scipy.optimize
res = scipy.optimize.minimize(fun, mom0_tensor.detach().numpy(),
                              method='BFGS',
                              jac=jac,
                              bounds=None,
                              tol=None,
                              callback=None,
                              options={
                                  'maxcor': 10,
                                  'ftol': 1.e-09,
                                  'gtol': 1e-03,
                                  'eps': 1e-08,
                                  'maxfun': 500,
                                  'maxiter': 100,
                                  'iprint': 1,
                                  'maxls': 25
                              })

In [ ]:
# setting parameters for optimization
reg_param = 10**-3 

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
mom0_tensor = torch.cat(H.module.manifold.unroll_cotan().copy(),0).requires_grad_().view(-1)
    
gd0_tensor.retain_grad()
mom0_tensor.retain_grad()
    
EnergyFunctional = model.EnergyFunctional(modules, H, constr_Id,source, target, 2, reg_param, attach)

# do registration
X = [gd0_tensor, mom0_tensor]
X = model.gradientdescent(EnergyFunctional, X)

In [ ]:
modules.manifold.cotan

In [ ]:
%matplotlib qt5
H.module.manifold.fill_gd(X[0])
H.module.manifold.fill_cotan(X[1])

states, controls = dm.shooting.shoot_euler(H, it=10)

import matplotlib.cm as cm
colors = cm.rainbow(np.linspace(0, 1, len(states)))

plt.figure()
for s,c in zip(states, colors):
    plt.scatter(s[0][-1].gd.view(-1,2)[:, 0].detach().numpy(), s[0][-1].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
    plt.scatter(s[1][-1].gd.view(-1,2)[:, 0].detach().numpy(), s[1][-1].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
    plt.scatter(s[2][0].gd.view(-1,2)[:, 0].detach().numpy(), s[2][0].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
    plt.scatter(s[2][1].gd.view(-1,2)[:, 0].detach().numpy(), s[2][1].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
plt.plot(target[0][:, 0].detach().numpy(), target[0][:, 1].detach().numpy(), 'xk')
plt.plot(target[1][:, 0].detach().numpy(), target[1][:, 1].detach().numpy(), 'xk')

plt.plot(source[0][:, 0].detach().numpy(), source[0][:, 1].detach().numpy(), '.k')
plt.plot(source[1][:, 0].detach().numpy(), source[1][:, 1].detach().numpy(), '.k')

    
plt.axis('equal')
plt.show()

In [ ]:
# Visualization of grid deformation for each submodule
nx, ny = 30, 30
sx, sy = 30, 30
x, y = torch.meshgrid([torch.arange(0, nx), torch.arange(0, ny)])
x = sx*(x.type(torch.DoubleTensor)/nx - 0.5)
y = sy*(y.type(torch.DoubleTensor)/ny - 0.5)

gridpoints = dm.usefulfunctions.grid2vec(x, y).type(torch.DoubleTensor)


phi = mm.largeDeformation(modules, states, controls, gridpoints)
x1, y1 = dm.usefulfunctions.vec2grid(phi[0], nx, ny)
x2, y2 = dm.usefulfunctions.vec2grid(phi[1], nx, ny)
x3, y3 = dm.usefulfunctions.vec2grid(phi[2], nx, ny)



mm.plot_grid( x1.detach().numpy(), y1.detach().numpy(), color = 'blue')
mm.plot_grid(x2.detach().numpy(), y2.detach().numpy(), color = 'blue')
mm.plot_grid( x3.detach().numpy(), y3.detach().numpy(), color = 'blue')
plt.show()


In [ ]:
def test_gradcheck_Hamiltonian():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
                
        return H()
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)


test_gradcheck_Hamiltonian()

In [ ]:
def test_gradcheck_EnergyFunctional():
    def energy(gd, mom):

        return model.EnergyFunctional(H.module, H, constr_Id,source, target, 2, reg_param).energy_tensor(gd, mom)
    return torch.autograd.gradcheck(energy, (gd, mom), raise_exception=True)

gd = gd0_tensor
mom = mom0_tensor


test_gradcheck_EnergyFunctional()

In [ ]:
def test_gradcheck_shooting():
    def shooting(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        
        intermediate_states, intermediate_controls = dm.shooting.shoot_euler(H, it=2)
        return H.module.manifold.unroll_cotan()
    return torch.autograd.gradcheck(shooting, (gd, mom), raise_exception=True,atol=1e-2)

test_gradcheck_shooting()

In [ ]:
def test_gradcheck_attachement():
    def attach(gd, mom):
        reg_param = 10**-3 

        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        energyfunc = model.EnergyFunctional(H.module, H, constr_Id,source, target, 2, reg_param)

        return energyfunc.attach() 
    return torch.autograd.gradcheck(attach, (gd, mom), raise_exception=True)

gd = torch.cat(H.module.manifold.unroll_gd(),0).requires_grad_().view(-1)
mom = torch.cat(H.module.manifold.unroll_cotan(),0).requires_grad_().view(-1)

test_gradcheck_attachement()

In [ ]:
def test_gradcheck_EnergyFunctional():
    def energy(gd, mom):
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.module.compute_geodesic_variables(constr_Id)
        return H.module.controls[0]
    return torch.autograd.gradcheck(energy, (gd, mom), raise_exception=True)

gd = torch.cat(H.module.manifold.unroll_gd(),0).requires_grad_().view(-1)
mom = torch.cat(H.module.manifold.unroll_cotan(),0).requires_grad_().view(-1)

test_gradcheck_EnergyFunctional()

In [ ]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        return mod.autoaction()
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = data.modules[0].copy()
gd = torch.cat(mod.manifold.gd.copy())
mom = torch.cat(mod.manifold.cotan.copy())

gd.requires_grad
mom.requires_grad

test_gradcheck_autoaction()

In [ ]:
modules.module_list[-1].autoaction()

In [ ]:
modules.background.costop_inv()

In [ ]:
torch.inverse(modules.background.K_q())

In [ ]:
modules.background.K_q() #- torch.eye(20)